In [1]:
# declare a list tasks whose products you want to use as inputs
upstream = ['fetch_securities']

In [2]:
# Parameters
upstream = {
    "fetch_securities": {
        "nb": "/Users/aiujdm2/market_watch/output/notebooks/fetch_securities.ipynb",
        "data": "/Users/aiujdm2/market_watch/output/data/raw/securities.csv",
    }
}
product = {
    "nb": "/Users/aiujdm2/market_watch/output/notebooks/normalize_security_names.ipynb",
    "data": "/Users/aiujdm2/market_watch/output/data/interim/normalized_security_names.csv",
}


In [3]:
import ast
import pandas as pd
import numpy as np
import json
from pathlib import Path
from src.utils import preprocess_text

import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /Users/aiujdm2/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aiujdm2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/aiujdm2/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/Users/aiujdm2/.local/share/virtualenvs/market_watch-dtlP-L11/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package wordnet to /Users/aiujdm2/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aiujdm2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Dow

True

In [4]:
securities_file_path = upstream['fetch_securities']['data']
securities_df = pd.read_csv(securities_file_path, index_col=0)

In [5]:
securities_df.drop_duplicates(subset=['cik'], inplace=True)

In [6]:
securities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9193 entries, 0 to 9192
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   cik                    9193 non-null   int64  
 1   ticker                 9193 non-null   object 
 2   exchanges              9193 non-null   object 
 3   ein                    9179 non-null   float64
 4   full_name              9193 non-null   object 
 5   former_names           9193 non-null   object 
 6   short_name             499 non-null    object 
 7   gics_sector            499 non-null    object 
 8   gics_sub_industry      499 non-null    object 
 9   sic                    8415 non-null   float64
 10  sic_description        8398 non-null   object 
 11  headquarters_location  499 non-null    object 
dtypes: float64(2), int64(1), object(9)
memory usage: 933.7+ KB


In [7]:
securities_df.former_names.replace('[]', np.nan, inplace=True)
securities_df[securities_df.former_names.isnull()]

,cik,ticker,exchanges,ein,full_name,former_names,short_name,gics_sector,gics_sub_industry,sic,sic_description,headquarters_location
0,1759425,MIRM,['Nasdaq'],831281555.0,"Mirum Pharmaceuticals, Inc.",NaN,NaN,NaN,NaN,2834.0,Pharmaceutical Preparations,NaN
1,1734262,CTK,['NYSE'],0.0,CooTek(Cayman)Inc.,NaN,NaN,NaN,NaN,7374.0,Services-Computer Processing & Data Preparation,NaN
6,1832161,KEYR,[''],850738656.0,KeyStar Corp.,NaN,NaN,NaN,NaN,5900.0,Retail-Miscellaneous Retail,NaN
7,1860742,BLCO,[''],0.0,Bausch & Lomb Corp,NaN,NaN,NaN,NaN,3851.0,Ophthalmic Goods,NaN
9,1034054,SBAC,['Nasdaq'],650716501.0,SBA COMMUNICATIONS CORP,NaN,SBA Communications,Real Estate,Specialized REITs,6798.0,Real Estate Investment Trusts,"Boca Raton, Florida"
...,...,...,...,...,...,...,...,...,...,...,...,...
9185,25232,CUZ,['NYSE'],580869052.0,COUSINS PROPERTIES INC,NaN,NaN,NaN,NaN,6798.0,Real Estate Investment Trusts,NaN
9186,65433,MXF,['NYSE'],133069854.0,MEXICO FUND INC,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9187,1832765,EAC,"['Nasdaq', 'Nasdaq', 'Nasdaq']",0.0,Edify Acquisition Corp.,NaN,NaN,NaN,NaN,6770.0,Blank Checks,NaN
9188,764630,USBL,['OTC'],61120072.0,UNITED STATES BASKETBALL LEAGUE INC,NaN,NaN,NaN,NaN,6794.0,Patent Owners & Lessors,NaN


In [8]:
def nomalize_names(securities_df):
    def extract_name(string):
        if len(str(string).strip()) > 5:
            string = ast.literal_eval(string)
            string = json.dumps(string)
            lst = json.loads(string)
            name = lst[0]["name"]
            return name
        else:
            return np.nan
            
    securities_df['former_name'] = securities_df.former_names.apply(extract_name)
    securities_df = securities_df.replace(to_replace=["/[A-Za-z]+/?"], value=[''], regex=True)
    securities_df = securities_df.replace(to_replace=[r'\\DE\\'], value=[''], regex=True)
    
    not_null_cond = ~securities_df['former_name'].isnull()
    
    securities_df.loc[not_null_cond,'former_name'] = preprocess_text(securities_df.loc[not_null_cond,'former_name'])
    
    securities_df['full_name'] = preprocess_text(securities_df.full_name)
    securities_df.replace('[]', np.nan, inplace=True)

    columns_to_select = ['cik','ticker','full_name', 'former_name']
    
    return securities_df[columns_to_select]

In [9]:
securities_df = nomalize_names(securities_df)

In [10]:
output_file_path = product['data']
Path(output_file_path).parent.mkdir(exist_ok=True, parents=True)
securities_df.to_csv(output_file_path)
print(f"Saved file {output_file_path}")

Saved file /Users/aiujdm2/market_watch/output/data/interim/normalized_security_names.csv
